In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Load data : reading the images and converting all to array

In [2]:
Datadirectory = "Data/"
Classes = ["Face_Mask", "No_Mask"]
img_size = 224
trainning_data = []

def create_training_data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category) # 0 1 # Label
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path,img)) #load image
                new_arr = cv2.resize(img_arr, (img_size, img_size)) #resize image
                trainning_data.append([new_arr,class_num])
            except Exception as e:
                pass

In [3]:
create_training_data()

## Merge, label, reshape data

In [4]:
# Merge data
import random
random.shuffle(trainning_data)

In [5]:
#label data
X = [] #data/features
y = [] #label

for features, label in trainning_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1,img_size,img_size,3)

In [6]:
X.shape

(1878, 224, 224, 3)

In [7]:
#Normalize the data
X = X/255.0

In [8]:
y = np.array(y)

## Save data

In [9]:
import pickle

pickle_out = open('X.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('y.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()

## Read data

In [10]:
pickle_in = open("X.pickle", 'rb')
X = pickle.load(pickle_in)

pickle_in = open('y.pickle', 'rb')
y = pickle.load(pickle_in)

# Deep learning model for training 

## Transfer Learning

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
model = tf.keras.applications.mobilenet.MobileNet()

2022-04-25 15:54:43.260710: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-25 15:54:43.260832: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [ ]:
model.summary()

In [13]:
base_input = model.layers[0].input

In [14]:
base_output = model.layers[-4].output

In [15]:
Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)
#final_output = layers.Activation('sigmoid')(final_output)

In [16]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [17]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

## Setting for binary classification (Face mask/ without mask)

In [18]:
new_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
new_model.fit(X, y, epochs=20, validation_split=0.1)

Epoch 1/20
53/53 [==============================] - 10s 192ms/step - loss: 0.5855 - accuracy: 0.9604 - val_loss: 0.6517 - val_accuracy: 0.9574
Epoch 2/20
53/53 [==============================] - 9s 174ms/step - loss: 0.9415 - accuracy: 0.9355 - val_loss: 1.0579 - val_accuracy: 0.9309
Epoch 3/20
53/53 [==============================] - 9s 174ms/step - loss: 0.5255 - accuracy: 0.9645 - val_loss: 0.6508 - val_accuracy: 0.9574
Epoch 4/20
53/53 [==============================] - 9s 172ms/step - loss: 0.2536 - accuracy: 0.9834 - val_loss: 0.5697 - val_accuracy: 0.9628
Epoch 5/20
53/53 [==============================] - 9s 174ms/step - loss: 0.2185 - accuracy: 0.9852 - val_loss: 0.5697 - val_accuracy: 0.9628
Epoch 6/20
53/53 [==============================] - 9s 172ms/step - loss: 0.3820 - accuracy: 0.9746 - val_loss: 7.4689 - val_accuracy: 0.5053
Epoch 7/20
53/53 [==============================] - 9s 172ms/step - loss: 0.2574 - accuracy: 0.9822 - val_loss: 0.6705 - val_accuracy: 0.9468
Epoch

In [21]:
new_model.save('face_mask.h5')